# [`S2FFT`](https://github.com/astro-informatics/s2fft) - __Wigner transform__ Interactive Tutorial
---



This tutorial demonstrates how to call the Wigner transform apis within `S2FFT`. Specifically we will be working with the forward and inverse Wigner transforms (see [McEwen *et al*](https://arxiv.org/pdf/1508.03101.pdf)). To demonstrate how to apply ``S2FFT`` transforms we must first construct an input signal which is correctly sampled on the rotation group, sadly no particularly appealing come to hand so we will be working with a random signal.

In [1]:
import numpy as np
import s2fft 

L = 128
N = 3
reality = True
rng = np.random.default_rng(0)
flmn = s2fft.utils.signal_generator.generate_flmn(rng, L, N, reality=reality)

/opt/homebrew/Caskroom/miniforge/base/envs/s2fft/lib/python3.9/site-packages/jax/_src/lib/__init__.py:34: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


### Computing the inverse Wigner transform
---
Lets run the JAX function to compute the inverse Wigner transform of this random signal.

In [2]:
f = s2fft.wigner.inverse_jax(flmn, L, N, reality=reality)

### Computing the Wigner transform
---
Lets run the JAX function to get us back to the random Wigner coefficients.

In [3]:
flmn_test = s2fft.wigner.forward_jax(f, L, N, reality=reality)

Lets check the roundtrip error, which should be close to machine precision for the McEwen-Wiaux sampling theorem which is selected by default

In [4]:
print(f"Mean absolute error = {np.nanmean(np.abs(flmn_test - flmn))}")

Mean absolute error = 4.994860670436926e-14
